# Rough

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib widget

In [29]:
import pandas as pd

import seaborn as sns

import numpy as np
import matplotlib.pyplot as plt

from xgboost import XGBRegressor
import optuna

from sklearn.metrics import mean_squared_error

import warnings

from poly2.utils import (
    get_dispersal_kernel, trait_vec, load_train_test_data,
    get_best_model, get_model_cv_score, HyperparamsObj,
    get_best_params, 
)
from cluster.hyper import train_test_scores, run_optuna
from poly2.config import Config

In [30]:
warnings.simplefilter(action='ignore', category=FutureWarning)

## Which model?

In [3]:
model = 'all'